In [30]:
# @title ライブラリのインポート
import pandas as pd
import numpy as np


!pip install statsmodels
!pip install linearmodels

import statsmodels.api as sm
from linearmodels.panel import PanelOLS
from linearmodels.panel import RandomEffects

# 線形回帰

In [31]:
# @title 結果のサマライズ用関数(linearmodels用)を作成
def summarize1(result,model_name):
    df = pd.DataFrame({'params': result.params, 'se': result.std_errors}).reset_index()
    df = df[df["index"].isin(["X1","X2","X3","const"])]
    df_long = df.melt(id_vars = ["index"])
    summary = df_long.sort_values(by=['index']).rename(columns={'index': 'variable_names','variable': 'Types_of_estimates','value': model_name})
    summary = summary.set_index(['variable_names','Types_of_estimates'])
    return summary

In [32]:
# @title 結果のサマライズ用関数を作成
def summarize(result,model_name):
    df = pd.DataFrame({'params': result.params, 'se': result.bse}).reset_index()
    df = df[df["index"].isin(["X1","X2","X3","const"])]
    df_long = df.melt(id_vars = ["index"])
    summary = df_long.sort_values(by=['index']).rename(columns={'index': 'variable_names','variable': 'Types_of_estimates','value': model_name})
    summary = summary.set_index(['variable_names','Types_of_estimates'])
    return summary

In [42]:
# @title パネルデータの生成

# データサイズの設定
n_individuals = 50  # 個体の数
n_time_periods = 20  # 時間の数
n_observations = n_individuals * n_time_periods  # 観測値の総数

# 個体固有の効果を生成
individual_effects = np.random.normal(0, 0.2, n_individuals)

# 説明変数を生成
X1 = np.random.normal(5, 2, n_observations)  # 説明変数1
X2 = np.random.normal(3, 1.5, n_observations)  # 説明変数2
X3 = np.random.normal(-2, 1, n_observations)  # 説明変数3

# 係数の設定
a = 2
b1 = 0.5
b2 = -0.3
b3 = 0.2

# 誤差項を生成
error_term = np.random.normal(0, 0.1, n_observations)

# 被説明変数の生成
Y = a + b1*X1 + b2*X2 + b3*X3 + individual_effects.repeat(n_time_periods) + error_term

# 説明変数と固定効果が相関するケースのデータ生成
X1_corr = X1 + individual_effects.repeat(n_time_periods)
data_corr = pd.DataFrame({
    'individual_id': np.repeat(np.arange(n_individuals), n_time_periods),
    'time': np.tile(np.arange(n_time_periods), n_individuals),
    'X1': X1_corr,
    'X2': X2,
    'X3': X3,
    'Y': Y
})

# 説明変数と固定効果が相関しないケースのデータ生成
data_no_corr = pd.DataFrame({
    'individual_id': np.repeat(np.arange(n_individuals), n_time_periods),
    'time': np.tile(np.arange(n_time_periods), n_individuals),
    'X1': X1,
    'X2': X2,
    'X3': X3,
    'Y': Y
})

# 正解データの作成

true = {
    'variable_names': ['X1', 'X2', 'X3',"const"],
    'Types_of_estimates': ['params', 'params', 'params','params'],
    'True_values': [0.5, -0.3, 0.2,2]
}
true = pd.DataFrame(true)
true = true.set_index(['variable_names','Types_of_estimates'])

# 表示
print("data_corr 固定効果と説明変数の相関があるデータ")
display(data_corr.head())
print("\n")
print("ndata_no_corr 固定効果と説明変数の相関がないデータ")
display(data_no_corr.head())

data_corr 固定効果と説明変数の相関があるデータ


,individual_id,time,X1,X2,X3,Y
0,0,0,5.661615,5.970970,-3.332439,2.409311
1,0,1,3.596733,-0.083577,-3.927312,3.023573
2,0,2,5.468701,2.239427,-3.068533,3.477383
3,0,3,4.053252,3.006220,-3.102142,2.525219
4,0,4,5.334407,3.180351,-3.181364,2.894961




ndata_no_corr 固定効果と説明変数の相関がないデータ


,individual_id,time,X1,X2,X3,Y
0,0,0,5.722143,5.970970,-3.332439,2.409311
1,0,1,3.657261,-0.083577,-3.927312,3.023573
2,0,2,5.529229,2.239427,-3.068533,3.477383
3,0,3,4.113781,3.006220,-3.102142,2.525219
4,0,4,5.394936,3.180351,-3.181364,2.894961


In [43]:
# 相関のあるデータ

# パネルデータ形式に変換
data = data_corr.set_index(['individual_id', 'time'])

# モデルの定義と推定
mod = PanelOLS.from_formula('Y ~ X1 + X2 + X3 + EntityEffects', data)
fixed_effects_res = mod.fit(cov_type='clustered', cluster_entity=True)

# 結果を格納
fe = summarize1(fixed_effects_res,"FE_model")

# モデルの定義と推定
mod = RandomEffects.from_formula('Y ~ X1 + X2 + X3', data)
random_effects_res = mod.fit(cov_type="clustered", cluster_entity=True)

# 結果を格納
re = summarize1(random_effects_res,"RE_model")

# 結果の比較
summary = pd.concat([fe,re,true],axis=1)
df_display = summary.fillna('')  # 'NaN' を空の文字列で置き換え
display(df_display)

FE_model  RE_model True_values
variable_names Types_of_estimates                                
X1             params              0.498766  0.549362         0.5
               se                  0.001496  0.002635            
X2             params             -0.299351  -0.24873        -0.3
               se                  0.002002  0.003596            
X3             params              0.198765  0.118802         0.2
               se                  0.002877  0.005332            
const          params                                         2.0

In [44]:
# 相関のないデータ

# パネルデータ形式に変換
data = data_no_corr.set_index(['individual_id', 'time'])

# モデルの定義と推定
mod = PanelOLS.from_formula('Y ~ 1 +X1 + X2 + X3 + EntityEffects', data)
fixed_effects_res = mod.fit(cov_type="clustered", cluster_entity=True)

# 結果を格納
fe = summarize1(fixed_effects_res,"FE_model")

# モデルの定義と推定
mod = RandomEffects.from_formula('Y ~ 1 + X1 + X2 + X3', data)
random_effects_res = mod.fit(cov_type="clustered", cluster_entity=True)

# 結果を格納
re = summarize1(random_effects_res,"RE_model")

# 結果の比較
summary = pd.concat([fe,re,true],axis=1)
df_display = summary.fillna('')  # 'NaN' を空の文字列で置き換え
display(df_display)

FE_model  RE_model True_values
variable_names Types_of_estimates                                
X1             params              0.498766  0.498765         0.5
               se                  0.001497  0.001502            
X2             params             -0.299351 -0.299311        -0.3
               se                  0.002003  0.001986            
X3             params              0.198765  0.198804         0.2
               se                  0.002878  0.002882            
const          params                                         2.0

# ロジスティック回帰

In [50]:
# @title パネルデータを生成

# データサイズの設定
n_individuals = 500  # 個体の数
n_time_periods = 30  # 時間の数
n_observations = n_individuals * n_time_periods  # 観測値の総数

# 個体固有の効果を生成
individual_effects = np.random.normal(0, 1, n_individuals)

# 説明変数を生成
X1 = np.random.normal(5, 2, n_observations)  # 説明変数1
X2 = np.random.normal(3, 1.5, n_observations)  # 説明変数2
X3 = np.random.normal(-2, 1, n_observations)  # 説明変数3

# 係数の設定
a = -1.2
b1 = 0.5
b2 = -0.3
b3 = 0.2

# 被説明変数の生成
linear_combination = a + b1*X1 + b2*X2 + b3*X3 + individual_effects.repeat(n_time_periods)
# ロジスティック関数を適用して確率を計算
probabilities = 1 / (1 + np.exp(-linear_combination))

# バイナリアウトカム（0または1）を生成
Y = np.random.binomial(1, probabilities)

# 説明変数と固定効果が相関するケースのデータ生成
X1_corr = X1 + individual_effects.repeat(n_time_periods)
data_corr = pd.DataFrame({
    'individual_id': np.repeat(np.arange(n_individuals), n_time_periods),
    'time': np.tile(np.arange(n_time_periods), n_individuals),
    'X1': X1_corr,
    'X2': X2,
    'X3': X3,
    'Y': Y
})

# 説明変数と固定効果が相関しないケースのデータ生成
data_no_corr = pd.DataFrame({
    'individual_id': np.repeat(np.arange(n_individuals), n_time_periods),
    'time': np.tile(np.arange(n_time_periods), n_individuals),
    'X1': X1,
    'X2': X2,
    'X3': X3,
    'Y': Y
})

true = {
    'variable_names': ['X1', 'X2', 'X3',"const"],
    'Types_of_estimates': ['params', 'params', 'params','params'],
    'True_values': [0.5, -0.3, 0.2,-1.2]
}
true = pd.DataFrame(true)
true = true.set_index(['variable_names','Types_of_estimates'])

# 表示
print("data_corr 固定効果と説明変数の相関があるデータ")
display(data_corr.head())
print("\n")
print("ndata_no_corr 固定効果と説明変数の相関がないデータ")
display(data_no_corr.head())

data_corr 固定効果と説明変数の相関があるデータ


,individual_id,time,X1,X2,X3,Y
0,0,0,3.544523,0.617867,-2.433868,1
1,0,1,4.805361,2.948917,-0.528066,0
2,0,2,2.199612,2.993050,-1.378040,0
3,0,3,7.367408,5.557159,-2.173563,1
4,0,4,10.162963,6.618245,-1.798844,1




ndata_no_corr 固定効果と説明変数の相関がないデータ


,individual_id,time,X1,X2,X3,Y
0,0,0,2.455091,0.617867,-2.433868,1
1,0,1,3.715928,2.948917,-0.528066,0
2,0,2,1.110179,2.993050,-1.378040,0
3,0,3,6.277976,5.557159,-2.173563,1
4,0,4,9.073531,6.618245,-1.798844,1


In [51]:
from statsmodels.discrete.conditional_models import ConditionalLogit
# 相関のあるデータ

# データ変換
data = data_corr

# 説明変数と目的変数を設定
X = data[['X1', 'X2', 'X3']]
Y = data['Y']
group = data['individual_id']

# 固定効果モデルの設定
model = ConditionalLogit(Y, X, groups=group)
# モデルのフィット
results = model.fit()

# 結果の表示
#print(results.summary())

# 結果を格納
cl = summarize(results,"model_conditional_logit")

# 説明変数（X）と目的変数（Y）の設定
X = pd.get_dummies(data[['X1', 'X2', 'X3', 'individual_id']],columns=['individual_id'])
Y = data['Y']

# ロジスティック回帰モデルを設定

model = sm.Logit(Y, X)

# モデルのフィット
results = model.fit()

# 結果の表示
#print(results.summary())

# 結果を格納
dl = summarize(results,"model_dummy_logit")

# 説明変数（X）と目的変数（Y）の設定
X = data[['X1', 'X2', 'X3']]
Y = data['Y']

# ロジスティック回帰モデルを設定
# statsmodelsでは、定数項を追加する必要があります
X = sm.add_constant(X)
model = sm.Logit(Y, X)

# モデルのフィット
results = model.fit()

# 結果の表示
#print(results.summary())

# 結果を格納
logit = summarize(results,"model_logit")

# 結果の比較
summary = pd.concat([cl,dl,logit,true],axis=1)
df_display = summary.fillna('')  # 'NaN' を空の文字列で置き換え
display(df_display)

/usr/local/lib/python3.10/dist-packages/statsmodels/discrete/conditional_models.py:80: UserWarning: Dropped 2 groups and 60 observations for having no within-group variance
  warnings.warn(msg)


         Current function value: 0.505973
         Iterations: 35
Optimization terminated successfully.
         Current function value: 0.541033
         Iterations 6


/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


model_conditional_logit model_dummy_logit  \
variable_names Types_of_estimates                                             
X1             params                            0.511712          0.531882   
               se                                0.011875          0.012152   
X2             params                           -0.283481         -0.294643   
               se                                0.013795          0.014075   
X3             params                            0.213677          0.222182   
               se                                0.019921          0.020316   
const          params                                                         
               se                                                             

                                   model_logit True_values  
variable_names Types_of_estimates                           
X1             params                 0.589581         0.5  
               se                     0.011111              
X2             params                -0.272870        -0.3  
               se                     0.013223              
X3             params                 0.202639         0.2  
               se                     0.019132              
const          params                -1.712577        -1.2  
               se                     0.074352

# ポアソン回帰

In [79]:
# @title パネルデータ生成

# データサイズの設定
n_individuals =300  # 個体の数
n_time_periods = 100  # 時間の数
n_observations = n_individuals * n_time_periods  # 観測値の総数

# 個体固有の効果を生成
individual_effects = np.random.normal(0, 1, n_individuals)

# 説明変数を生成
X1 = np.random.normal(5, 2, n_observations)  # 説明変数1
X2 = np.random.normal(3, 1.5, n_observations)  # 説明変数2
X3 = np.random.normal(-2, 1, n_observations)  # 説明変数3

# 係数の設定
a = -1.2
b1 = 0.5
b2 = -0.3
b3 = 0.2

# 被説明変数の生成
lam = np.exp(a + b1*X1 + b2*X2 + b3*X3 + individual_effects.repeat(n_time_periods))

# ポアソンアウトカムを生成
Y = np.random.poisson(lam)

# 説明変数と固定効果が相関するケースのデータ生成
X1_corr = X1 + individual_effects.repeat(n_time_periods)
data_corr = pd.DataFrame({
    'individual_id': np.repeat(np.arange(n_individuals), n_time_periods),
    'time': np.tile(np.arange(n_time_periods), n_individuals),
    'X1': X1_corr,
    'X2': X2,
    'X3': X3,
    'Y': Y
})

# 説明変数と固定効果が相関しないケースのデータ生成
data_no_corr = pd.DataFrame({
    'individual_id': np.repeat(np.arange(n_individuals), n_time_periods),
    'time': np.tile(np.arange(n_time_periods), n_individuals),
    'X1': X1,
    'X2': X2,
    'X3': X3,
    'Y': Y
})

true = {
    'variable_names': ['X1', 'X2', 'X3',"const"],
    'Types_of_estimates': ['params', 'params', 'params','params'],
    'True_values': [0.5, -0.3, 0.2,-1.2]
}
true = pd.DataFrame(true)
true = true.set_index(['variable_names','Types_of_estimates'])

# 表示
print("data_corr 固定効果と説明変数の相関があるデータ")
display(data_corr.head())
print("\n")
print("ndata_no_corr 固定効果と説明変数の相関がないデータ")
display(data_no_corr.head())

data_corr 固定効果と説明変数の相関があるデータ


,individual_id,time,X1,X2,X3,Y
0,0,0,4.118820,4.561318,-1.982326,0
1,0,1,6.956971,3.362066,-2.276380,2
2,0,2,2.640285,3.946246,-2.808157,0
3,0,3,6.180078,3.086957,-2.389497,0
4,0,4,5.364913,3.124906,-0.259857,2




ndata_no_corr 固定効果と説明変数の相関がないデータ


,individual_id,time,X1,X2,X3,Y
0,0,0,4.385777,4.561318,-1.982326,0
1,0,1,7.223929,3.362066,-2.276380,2
2,0,2,2.907242,3.946246,-2.808157,0
3,0,3,6.447036,3.086957,-2.389497,0
4,0,4,5.631871,3.124906,-0.259857,2


In [80]:
from statsmodels.discrete.conditional_models import ConditionalPoisson
# 相関のあるデータ

# データ変換
data = data_corr

# 条件付き最尤法--------------------------------------------------------------------------------------------------------------
# 説明変数と目的変数を設定
X = data[['X1', 'X2', 'X3']]
Y = data['Y']
group = data['individual_id']

# 固定効果モデルの設定
model = ConditionalPoisson(Y, X, groups=group)
# モデルのフィット
results = model.fit()

# 結果の表示
#print(results.summary())

# 結果を格納
cp = summarize(results,"model_conditional_poisson")

# ダミー変数最尤法--------------------------------------------------------------------------------------------------------------
# 説明変数（X）と目的変数（Y）の設定
X = pd.get_dummies(data[['X1', 'X2', 'X3', 'individual_id']],columns=['individual_id'],drop_first=True)
Y = data['Y']

# ポアソン回帰モデルを設定
model = sm.Poisson(Y, X)

# モデルのフィット
results = model.fit()

# 結果の表示
#print(results.summary())

# 結果を格納
dp = summarize(results,"model_dummy_poisson")

# 最尤法--------------------------------------------------------------------------------------------------------------
# 説明変数（X）と目的変数（Y）の設定
X = data[['X1', 'X2', 'X3']]
Y = data['Y']

# ポアソン回帰モデルを設定
# statsmodelsでは、定数項を追加する必要があります
X = sm.add_constant(X)
model = sm.Poisson(Y, X)

# モデルのフィット
results = model.fit()

# 結果の表示
#print(results.summary())

# 結果を格納
poisson = summarize(results,"model_poisson")

# 結果の比較
summary = pd.concat([cp,dp,poisson,true],axis=1)
df_display = summary.fillna('')  # 'NaN' を空の文字列で置き換え
display(df_display)

Optimization terminated successfully.
         Current function value: 1.434201
         Iterations 32
Optimization terminated successfully.
         Current function value: 1.760779
         Iterations 7


model_conditional_poisson  \
variable_names Types_of_estimates                             
X1             params                               0.49796   
               se                                  0.001574   
X2             params                             -0.303683   
               se                                  0.002083   
X3             params                              0.202098   
               se                                  0.003142   
const          params                                         
               se                                             

                                  model_dummy_poisson  model_poisson  \
variable_names Types_of_estimates                                      
X1             params                        0.491625       0.590819   
               se                             0.00153       0.001369   
X2             params                       -0.307237      -0.308408   
               se                             0.00207       0.002054   
X3             params                         0.20758       0.204244   
               se                            0.003117       0.003087   
const          params                                      -1.436098   
               se                                           0.013545   

                                  True_values  
variable_names Types_of_estimates              
X1             params                     0.5  
               se                              
X2             params                    -0.3  
               se                              
X3             params                     0.2  
               se                              
const          params                    -1.2  
               se